In [1]:
import pandas as pd 
import numpy as np 
import requests
import json
import dotenv
import os

import sys
sys.tracebacklimit = 0

## Spotify Scraping

In [11]:
dotenv.load_dotenv()
spotify_client_id = os.getenv('spotify_clientid')
spotify_client_secret = os.getenv('spotify_clientsecret')


url = 'https://accounts.spotify.com/api/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded',}
data = {
    'grant_type': 'client_credentials',
    'client_id': spotify_client_id,
    'client_secret': spotify_client_secret,
}

response = requests.post(url, headers=headers, data=data)

token_info = response.json()
access_token = token_info['access_token']

In [27]:
playlist_id = "37i9dQZF1E4oJSdHZrVjxD"

url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
headers = {'Authorization': f'Bearer {access_token}'}

response = requests.get(url, headers=headers)
df = pd.json_normalize(json.loads(response.text), record_path=['items'])
# df.to_csv("scraped.csv")
# df


In [ ]:
artist_id = "4YRxDV8wJFPHPTeXepOstw"
options = "appears_on"
limit = "50"
url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
total_album_no = json.loads(response.content)['total']

main_dict = {}
print(np.arange(0,total_album_no,50))
for i in np.arange(0,total_album_no,50):
    print(i)
    offset = i
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers) 

    df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    main_dict[i] = df
main_df = pd.concat(main_dict.values())


[   0   50  100  150  200  250  300  350  400  450  500  550  600  650
  700  750  800  850  900  950 1000 1050 1100 1150 1200]
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200


In [60]:
main_df['Album_ID'] = main_df['external_urls.spotify'].apply(lambda x: x.split("/")[-1])
main_df['Producer'] = main_df['artists'].apply(lambda x: x[0]['name'])
main_df['Producer_ID'] = main_df['artists'].apply(lambda x: x[0]['id'])
main_df.to_csv("albums.csv")
# df


In [61]:
main_df

,album_type,total_tracks,href,id,images,name,release_date,release_date_precision,type,uri,artists,album_group,external_urls.spotify,Album_ID,Producer,Producer_ID
0,album,12,https://api.spotify.com/v1/albums/0Rkv5iqjF2ue...,0Rkv5iqjF2uenfL0OVB8hg,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Aashiqui 2,2013-04-06,day,album,spotify:album:0Rkv5iqjF2uenfL0OVB8hg,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0Rkv5iqjF2uenfL...,0Rkv5iqjF2uenfL0OVB8hg,Mithoon,09UmIX92EUH9hAK4bxvHx6
1,album,9,https://api.spotify.com/v1/albums/2Lxoc72vRTGd...,2Lxoc72vRTGdQfMvj7Ovi1,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Yeh Jawaani Hai Deewani,2013-03-30,day,album,spotify:album:2Lxoc72vRTGdQfMvj7Ovi1,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2Lxoc72vRTGdQfM...,2Lxoc72vRTGdQfMvj7Ovi1,Pritam,1wRPtKGflJrBx9BmLsSwlU
2,album,8,https://api.spotify.com/v1/albums/0a183xiCHiC1...,0a183xiCHiC1GQd8ou7WXO,[{'url': 'https://i.scdn.co/image/ab67616d0000...,ANIMAL,2023-11-24,day,album,spotify:album:0a183xiCHiC1GQd8ou7WXO,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0a183xiCHiC1GQd...,0a183xiCHiC1GQd8ou7WXO,Manan Bhardwaj,3pQ4aA7dkolyjUAMrVScgh
3,album,16,https://api.spotify.com/v1/albums/0LcyzKKw3RjF...,0LcyzKKw3RjFKL6ygISTeU,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Ek Tha Raja,2024-03-18,day,album,spotify:album:0LcyzKKw3RjFKL6ygISTeU,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0LcyzKKw3RjFKL6...,0LcyzKKw3RjFKL6ygISTeU,Badshah,0y59o4v8uw5crbN9M3JiL1
4,album,6,https://api.spotify.com/v1/albums/7sVzilmsqYcL...,7sVzilmsqYcLMYw2gtvoZM,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Hamari Adhuri Kahani (Original Motion Picture ...,2015-05-21,day,album,spotify:album:7sVzilmsqYcLMYw2gtvoZM,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/7sVzilmsqYcLMYw...,7sVzilmsqYcLMYw2gtvoZM,Jeet Gannguli,2kkQthS9OLpK4UqNWYqoVl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,compilation,11,https://api.spotify.com/v1/albums/6pA6wfGOTd4S...,6pA6wfGOTd4SOl7C8DFLie,[{'url': 'https://i.scdn.co/image/ab67616d0000...,"Open Stage Sessions, Vol. 39",2024-08-13,day,album,spotify:album:6pA6wfGOTd4SOl7C8DFLie,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/6pA6wfGOTd4SOl7...,6pA6wfGOTd4SOl7C8DFLie,Various Artists,0LyfQWJT6nXafLPZqxe9Of
34,compilation,10,https://api.spotify.com/v1/albums/0vLCAjawZPbM...,0vLCAjawZPbMtyhR6TcGED,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Dastak - A Bouquet Of Ghazals,2015-01-01,day,album,spotify:album:0vLCAjawZPbMtyhR6TcGED,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0vLCAjawZPbMtyh...,0vLCAjawZPbMtyhR6TcGED,Various Artists,0LyfQWJT6nXafLPZqxe9Of
35,compilation,7,https://api.spotify.com/v1/albums/35MkN8qsxx5K...,35MkN8qsxx5KEELxxISsak,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Celebrating World Music Day With Indipop Chart...,2015-06-18,day,album,spotify:album:35MkN8qsxx5KEELxxISsak,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/35MkN8qsxx5KEEL...,35MkN8qsxx5KEELxxISsak,Various Artists,0LyfQWJT6nXafLPZqxe9Of
36,compilation,1,https://api.spotify.com/v1/albums/2obuaVKDXzbu...,2obuaVKDXzbuTNU9Tx0HfK,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,day,album,spotify:album:2obuaVKDXzbuTNU9Tx0HfK,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2obuaVKDXzbuTNU...,2obuaVKDXzbuTNU9Tx0HfK,Various Artists,0LyfQWJT6nXafLPZqxe9Of


In [ ]:
all_albums = {}
for i,album_id in enumerate(main_df['Album_ID'].unique()):
    print(i,len(main_df))
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    response = requests.get(url, headers=headers) 
    
    df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    df_t['Album_ID'] = album_id

    df_t['Artist Song'] = df_t['artists'].apply(lambda x: artist_id in str(x))
    all_albums[album_id] = df_t

df_tracks = pd.concat(all_albums.values())
df_tracks.to_csv("tracks.csv")



0 1238
1 1238
2 1238
3 1238
4 1238
5 1238
6 1238
7 1238
8 1238
9 1238
10 1238
11 1238
12 1238
13 1238
14 1238
15 1238
16 1238
17 1238
18 1238
19 1238
20 1238
21 1238
22 1238
23 1238
24 1238
25 1238
26 1238
27 1238
28 1238
29 1238
30 1238
31 1238
32 1238
33 1238
34 1238
35 1238
36 1238
37 1238
38 1238
39 1238
40 1238
41 1238
42 1238
43 1238
44 1238
45 1238
46 1238
47 1238
48 1238
49 1238
50 1238
51 1238
52 1238
53 1238
54 1238
55 1238
56 1238
57 1238
58 1238
59 1238
60 1238
61 1238
62 1238
63 1238
64 1238
65 1238
66 1238
67 1238
68 1238
69 1238
70 1238
71 1238
72 1238
73 1238
74 1238
75 1238
76 1238
77 1238
78 1238
79 1238
80 1238
81 1238
82 1238
83 1238
84 1238
85 1238
86 1238
87 1238
88 1238
89 1238
90 1238
91 1238
92 1238
93 1238
94 1238
95 1238
96 1238
97 1238
98 1238
99 1238
100 1238
101 1238
102 1238
103 1238
104 1238
105 1238
106 1238
107 1238
108 1238
109 1238
110 1238
111 1238
112 1238
113 1238
114 1238
115 1238
116 1238
117 1238
118 1238
119 1238
120 1238
121 1238
122 1238
123

In [64]:
artist_songs = df_tracks[df_tracks['Artist Song']==True]
artist_songs = artist_songs[['artists','duration_ms','id','name','Album_ID']].rename(columns = {'name':"Track_Name",
                                                                                                'artists':"Track_Artists"})

album_details = main_df[['total_tracks','name','release_date','artists','Album_ID','Producer','Producer_ID']].rename(columns = {'name':'Album_Name',
                                                                                                                                'artists':"Album_Artists"})

final = artist_songs.merge(album_details, on = "Album_ID", how = "left", indicator=True)
final._merge.value_counts()

_merge
both          2050
left_only        0
right_only       0
Name: count, dtype: int64

In [67]:
final.to_csv("All_Details.csv", index = False)
final

,Track_Artists,duration_ms,id,Track_Name,Album_ID,total_tracks,Album_Name,release_date,Album_Artists,Producer,Producer_ID,_merge
0,[{'external_urls': {'spotify': 'https://open.s...,261974,56zZ48jdyY2oDXHVnwg5Di,Tum Hi Ho,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
1,[{'external_urls': {'spotify': 'https://open.s...,304899,6UgcN95w7vQxkR8sEFmwHG,Chahun Main Ya Naa,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
2,[{'external_urls': {'spotify': 'https://open.s...,306511,55eUtE9v3uatR0y0NijM6N,Hum Mar Jayenge,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
3,[{'external_urls': {'spotify': 'https://open.s...,266069,3E9ryvrKP3hUWX79PAakH2,Meri Aashiqui,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
4,[{'external_urls': {'spotify': 'https://open.s...,214470,06zJUeKU5KFy50Cho6pDcr,Aasan Nahin Yahan,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
2045,[{'external_urls': {'spotify': 'https://open.s...,284055,6LWzb6xYfQl1h4ixihYv17,Sun Bhi Le - Lofi,6pA6wfGOTd4SOl7C8DFLie,11,"Open Stage Sessions, Vol. 39",2024-08-13,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2046,[{'external_urls': {'spotify': 'https://open.s...,317413,0zciOy8kgToHnOc7m3RrAQ,Rishton Ke Saare Manzar,0vLCAjawZPbMtyhR6TcGED,10,Dastak - A Bouquet Of Ghazals,2015-01-01,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2047,[{'external_urls': {'spotify': 'https://open.s...,317348,21Nv3TlVXps943IGFWuJ8S,Rishton Ke Saare Manzar,35MkN8qsxx5KEELxxISsak,7,Celebrating World Music Day With Indipop Chart...,2015-06-18,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2048,[{'external_urls': {'spotify': 'https://open.s...,50749,3zk5QbltmuZWH8VapbdP4N,Phir Aur Kya Chahiye - Urban Drill Mix,2obuaVKDXzbuTNU9Tx0HfK,1,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both


In [ ]:
json.loads(response.text)
all_tracks = []    
data = response.json()
for item in data['items']:
    track = item['track']
    if track:
        # print((track['artists']))
        if track['artists'][0]['name'] != "Arijit Singh":continue
        if len(track['artists'])<=1: continue

        track_info = {
            'title': track['name'],
            'singer':'Arijit Singh',
            # 'singer': track['artists'][1] if len(track['artists'])>0 else track['artists'][0],
            'music_director': track['artists'][1]['name'],
            'release_date': track['album']['release_date'] 
        }
        all_tracks.append(track_info)

all_tracks

In [ ]:
track['artists']

Artist Search

In [ ]:
artist_id = "4YRxDV8wJFPHPTeXepOstw"

url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks'
headers = {'Authorization': f'Bearer {access_token}'}

response = requests.get(url, headers=headers)
json.loads(response.text)

## Genius (LyricsGenius) Method

In [2]:
import lyricsgenius
genius_access_token = os.getenv('geniusclientaccesstoken')
genius = lyricsgenius.Genius(genius_access_token)


In [10]:
artist = genius.search_artist("Arijit Singh", max_songs=50, sort="popularity")

Searching for songs by Arijit Singh...

Song 1: "Satranga"
Song 2: "Chal Wahan Jaate Hain"
Song 3: "Humdard"
Song 4: "Love Me Thoda Aur"
Song 5: "Tujhe Kitna Chahne Lage"
Song 6: "Laal Ishq"
Song 7: "Sawan Aaya Hai"
Song 8: "Phir Aur Kya Chahiye"
Song 9: "Tu Hi Hai Aashiqui (Solo)"
Song 10: "Palat - Tera Hero Idhar Hai"
Song 11: "Suno Na Sangemarmar"
Song 12: "Zaalima"
Song 13: "Kabhi Jo Baadal Barse"
Song 14: "Tera Yaar Hoon Main"
Song 15: "Thoda Aur"
Song 16: "Uska Hi Banana"
Song 17: "Raabta"
Song 18: "Abar Phire Ele"
Song 19: "Roke Na Ruke Naina"
Song 20: "Kalank - Title Track"
Song 21: "Aaj Se Teri"
Song 22: "Aaj Phir (From ”Hate Story 2")"
Song 23: "Pal Pal Dil Ke Paas - Title Track"
Song 24: "Saawali Si Raat"
Song 25: "Tomake Chai"
Song 26: "Lambiyaan Si Judaiyaan"
Song 27: "Aar Kobe?"
Song 28: "Dekho aloy alo aakash"
Song 29: "O Bedardeya"
Song 30: "Har Har Gange"
Song 31: "Main Dhoondne Ko Zamaane Mein"
Song 32: "Safar"
Song 33: "Mast Magan"
Song 34: "Thodi Jagah"
Song 35: "Kh

In [ ]:
artist.song("Satranga")

TypeError: the JSON object must be str, bytes or bytearray, not Song

## Wiki API Method

In [36]:
from bs4 import BeautifulSoup
dotenv.load_dotenv()
wiki_client_id = os.getenv('wiki_clientid')
wiki_client_secret = os.getenv('wiki_clientsecret')
wiki_access_token = os.getenv('wiki_accesstoken')

url = "https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Arijit_Singh"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table',class_="wikitable")

k=1
tabled = {}
tabled2 = {}
for table in tables:
    k+=1
    print(k)
    rows = table.find_all("tr")
    header = [i.text for i in rows[0]]
    nrows = [[i.text for i in row] for row in rows[1:]]

    print(header)
    print(nrows)
        

    tabled2[k] = (header, nrows)
    # tbl_i = pd.DataFrame(nrows, columns = header)
    # print(tbl_i)
    # tabled[k] = tbl_i



# table_df = pd.concat(tabled.values())
        # if item.get('scope')=="col": continue
        # print(item.scope)
        # raise Exception()
        # if item.scope == "col": print('col')
        # temp = item.get_text().split("\n")
        # values.append(temp)
    # values.append("LOL")


# put all the data into a dataframe
# df = pd.DataFrame(data = values)

2
['\n', 'Film\n', '\n', 'No\n', '\n', 'Song\n', '\n', 'Composer(s)\n', '\n', 'Writer(s)\n', '\n', 'Co-Singer(s)\n', '\n', 'Ref.\n', '\n', 'Note\n']
[['\n', 'Murder 2\n', '\n', '1\n', '\n', '"Phir Mohabbat"\n', '\n', 'Mithoon\n', '\n', 'Sayeed Quadri\n', '\n', 'Mohd. Irfan, Saim Bhatt\n', '\n', '[28]\n', '\n', '\n']]
3
['\n', 'Film\n', '\n', 'No\n', '\n', 'Song\n', '\n', 'Composer(s)\n', '\n', 'Writer(s)\n', '\n', 'Co-Singer(s)\n', '\n', 'Ref.\n', '\n', 'Note\n']
[['\n', 'Players\n', '\n', '2\n', '\n', '"Jhoom Jhoom Ta Hun Main"(Film Version)\n', '\n', 'Pritam Chakraborty\n', '\n', 'Ashish Pandit\n', '\n', '\n', '\n', '[28]\n', '\n', '\n'], ['\n', 'Agent Vinod\n', '\n', '3\n', '\n', '"Raabta"\n', '\n', 'Amitabh Bhattacharya\n', '\n', '[2]\n', '\n', '\n'], ['\n', '4\n', '\n', '"Raabta" (Kehte Hain Khuda Ne)\n', '\n', 'Shreya Ghoshal\n', '\n', '\n'], ['\n', '5\n', '\n', '"Raabta"(Night In a Motel)\n', '\n', 'Aditi Singh Sharma\n', '\n', '\n'], ['\n', '6\n', '\n', '"Raabta"(Siyaah Raatein